Parse TLE to KeplerInputs

In [1]:
import sys
sys.path.append('/workspaces/ma-max-wendler/scripts/keplertraces')  # Add the directory to the search path

In [2]:
import tleparse

In [3]:
# inputs = tleparse($tlespath$), i.e.
path = "/workspaces/ma-max-wendler/scripts/keplertraces/tles/examples_requested_at_2023-10-23-11-06-08/iridium-NEXT_2023-10-22-21-39-19.txt"
tles = tleparse.read(path)

In [4]:
# objects of the TLE-tool that among other things parses TLE values
tle = tles[0]
tle

TLE(name='IRIDIUM 106', norad='41917', classification='U', int_desig='17003A', epoch_year=2023, epoch_day=295.91774854, dn_o2=1.91e-06, ddn_o6=0.0, bstar=6.0989e-05, set_num=999, inc=86.3949, raan=133.2138, ecc=0.0001982, argp=93.4819, M=266.6603, n=14.34216513, rev_num=35442)

For comparison: IRIDIUM 106 TLE

```
IRIDIUM 106
1 41917U 17003A   23295.91774854  .00000191  00000+0  60989-4 0  9995
2 41917  86.3949 133.2138 0001982  93.4819 266.6603 14.34216513354425
```

Convert to KeplerInputs instance, contains two conversions:

In [5]:
from poliastro.twobody.angles import M_to_E, E_to_nu
import astropy.units as u
from math import pi

In [6]:
# mean anomaly to true anomaly via poliastro functions
ecc = u.Quantity(tle.ecc, unit=u.one)
mean_anom = u.Quantity(tle.M, unit=u.deg)
ecc, mean_anom

(<Quantity 0.0001982>, <Quantity 266.6603 deg>)

In [7]:
mean_anom_rad = mean_anom.to(u.rad)
# mean_anom_rad_negpos = u.Quantity(pi,unit=u.rad) - mean_anom.to(u.rad)
# -pi (when pi passed) + (mean_anom_rad - pi) (overlap over pi)
mean_anom_rad_negpos = mean_anom.to(u.rad) - u.Quantity(2*pi,unit=u.rad) 

mean_anom_rad, mean_anom_rad_negpos, 2*pi + mean_anom_rad_negpos.value

(<Quantity 4.65410022 rad>, <Quantity -1.62908509 rad>, 4.654100219355835)

In [8]:
import tletools

In [9]:
"""
**Warning**

    The mean anomaly must be between -π and π radians.
    The eccentricity must be less than 1.
"""
    
# tle-tools-based true anomaly
true_anom_t = u.Quantity(tletools.utils.M_to_nu(mean_anom_rad_negpos, tle.ecc), unit=u.rad).to(u.deg)
true_anom_t, true_anom_t.to(u.rad)

(<Quantity -93.36237315 deg>, <Quantity -1.62948081 rad>)

In [10]:
# poliastro based True anomaly -> can use mean anomaly in degrees
true_anom_p = E_to_nu( M_to_E( mean_anom_rad_negpos, ecc ), ecc )
_true_anom_p = E_to_nu( M_to_E( mean_anom_rad, ecc ), ecc )
true_anom_p, _true_anom_p, true_anom_p.to(u.deg)

(<Quantity -1.62948081 rad>,
 <Quantity -1.62948081 rad>,
 <Quantity -93.36237315 deg>)

In [11]:
from orbital.utilities import true_anomaly_from_mean

In [12]:
# OrbitalPy based True anomaly
# uses radian in 0,2pi range (Mnorm = fmod(M, 2 * pi))
true_anom_o = true_anomaly_from_mean(ecc.value, mean_anom_rad.value)
true_anom_o, u.Quantity( -1 * (2*pi - true_anom_o), unit=u.rad).to(u.deg)

(4.653704498287206, <Quantity -93.36237315 deg>)

In [13]:
import rebound

In [14]:
# rebound-based True anomaly
# can't really tell in which range M should be, as its C code
true_anom_r = rebound.M_to_f(tle.ecc, mean_anom_rad_negpos.value)
true_anom_r, u.Quantity(-360, unit=u.deg) + u.Quantity(true_anom_r, unit=u.rad).to(u.deg)

(4.653704498287205, <Quantity -93.36237315 deg>)

In [15]:
from skyfield import keplerlib

In [16]:
# can take mean anomalies in both ranges
ecc_anom = keplerlib.eccentric_anomaly(tle.ecc, mean_anom_rad.value)
_ecc_anom = keplerlib.eccentric_anomaly(tle.ecc, mean_anom_rad_negpos.value)
true_anom_s = keplerlib.true_anomaly_closed(tle.ecc, ecc_anom)
_true_anom_s = keplerlib.true_anomaly_closed(tle.ecc, _ecc_anom)

true_anom_s, u.Quantity(true_anom_s, unit=u.rad).to(u.deg), _true_anom_s

(-1.629480808892381, <Quantity -93.36237315 deg>, -1.629480808892381)

Semi-major axis

In [17]:
# tletools 
tle.a

7155.808268901222

In [18]:
from astropy.constants import GM_earth

In [20]:
# researched formula
secs_per_revolution = 86400 / tle.n
semimajoraxis = u.Quantity((secs_per_revolution**2 * GM_earth.value / (4 * pi**2) )**(1/3) / 1000, unit=u.km)
semimajoraxis

<Quantity 7155.80801877 km>